# 虚假新闻检测 - 特征工程

本notebook实现特征工程流程，包括：

1. 文本特征
   - TF-IDF特征
   - Word2Vec词向量
   - BERT预训练特征

2. 用户特征
   - 用户统计特征
   - 用户历史行为特征
   - 用户信誉度特征

3. 传播特征
   - 转发量和速度
   - 传播深度和广度
   - 时间序列特征

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
from pathlib import Path
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # 对于macOS
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

## 1. 数据加载与预处理

首先加载原始数据集，并进行基本的文本预处理：
1. 加载谣言和非谣言数据
2. 文本清洗（去除URL、@用户名、特殊字符等）
3. 中文分词
4. 加载用户和传播相关特征

In [ ]:
# 定义数据路径
DATA_DIR = Path('../data')
RUMORS_FILE = DATA_DIR / 'rumors_v170613.json'
CED_DATASET = DATA_DIR / 'CED_Dataset'

# 加载谣言数据
def load_rumors():
    rumors = []
    with open(RUMORS_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            rumors.append(json.loads(line))
    return pd.DataFrame(rumors)

# 加载CED数据集
def load_ced_dataset():
    # 加载原始微博
    original_path = CED_DATASET / 'original-microblog'
    originals = []
    for file in tqdm(list(original_path.glob('*.json')), desc='Loading original posts'):
        with open(file, 'r', encoding='utf-8') as f:
            originals.append(json.load(f))
    
    return pd.DataFrame(originals)

# 文本预处理
def preprocess_text(text):
    # 移除URL
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # 移除@用户名
    text = re.sub(r'@[\w\-]+', '', text)
    # 移除特殊字符和表情
    text = re.sub(r'[^\w\s\u4e00-\u9fff]', '', text)
    # 移除多余空格
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 加载停用词
def load_stopwords():
    stopwords = set(['的', '了', '是', '在', '我', '有', '和', '就', '不', '人', '都', '一', '一个', '上', '也', '很', '到', '说', '要', '去', '你', '会', '着', '没有', '看', '好', '自己', '这'])
    return stopwords

# 中文分词
def tokenize(text, stopwords):
    words = jieba.lcut(text)
    return [w for w in words if w not in stopwords and len(w) > 1]

print("加载数据...")
rumors_df = load_rumors()
ced_df = load_ced_dataset()

print("预处理文本...")
stopwords = load_stopwords()

# 预处理谣言数据
rumors_df['clean_text'] = rumors_df['rumorText'].apply(preprocess_text)
rumors_df['tokens'] = rumors_df['clean_text'].apply(lambda x: tokenize(x, stopwords))

# 预处理CED数据
ced_df['clean_text'] = ced_df['text'].apply(preprocess_text)
ced_df['tokens'] = ced_df['clean_text'].apply(lambda x: tokenize(x, stopwords))

print(f"谣言数据集大小: {len(rumors_df)}")
print(f"CED数据集大小: {len(ced_df)}")

# 显示预处理后的样例
print("\n预处理后的样例：")
print(rumors_df[['rumorText', 'clean_text', 'tokens']].head(1))

## 2. 文本特征提取

我们将实现三种不同的文本特征提取方法：
1. TF-IDF特征：捕捉词频-逆文档频率特征
2. Word2Vec词向量：学习词的分布式表示
3. BERT预训练特征：利用预训练模型提取上下文感知的特征

In [ ]:
# 2.1 TF-IDF特征提取
print("提取TF-IDF特征...")
tfidf = TfidfVectorizer(max_features=5000)
# 将分词后的文本转换为字符串
text_for_tfidf = rumors_df['tokens'].apply(lambda x: ' '.join(x))
tfidf_features = tfidf.fit_transform(text_for_tfidf)

print(f"TF-IDF特征维度: {tfidf_features.shape}")
print("\nTop 20 特征词:")
feature_names = tfidf.get_feature_names_out()
print(feature_names[:20])

# 2.2 Word2Vec词向量
print("\n训练Word2Vec模型...")
# 准备训练数据
sentences = rumors_df['tokens'].tolist() + ced_df['tokens'].tolist()

# 训练Word2Vec模型
w2v_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4)

# 获取文档向量（取所有词向量的平均）
def get_document_vector(tokens):
    vectors = []
    for token in tokens:
        if token in w2v_model.wv:
            vectors.append(w2v_model.wv[token])
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

w2v_features = np.array([get_document_vector(tokens) for tokens in rumors_df['tokens']])
print(f"Word2Vec特征维度: {w2v_features.shape}")

# 2.3 BERT特征提取
print("\n提取BERT特征...")
# 加载预训练模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')
model.eval()

def get_bert_features(texts, batch_size=32):
    features = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        # 编码文本
        encoded = tokenizer.batch_encode_plus(
            batch_texts,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        with torch.no_grad():
            outputs = model(**encoded)
            # 使用[CLS]标记的输出作为文档表示
            batch_features = outputs[0][:, 0, :].numpy()
            features.append(batch_features)
    
    return np.concatenate(features)

# 提取BERT特征（使用一小部分数据作为示例）
sample_texts = rumors_df['clean_text'].head(100).tolist()
bert_features = get_bert_features(sample_texts)
print(f"BERT特征维度: {bert_features.shape}")

# 可视化特征分布
plt.figure(figsize=(15, 5))

# TF-IDF特征分布
plt.subplot(131)
plt.hist(tfidf_features.data, bins=50)
plt.title('TF-IDF特征分布')
plt.xlabel('特征值')
plt.ylabel('频率')

# Word2Vec特征分布
plt.subplot(132)
plt.hist(w2v_features.flatten(), bins=50)
plt.title('Word2Vec特征分布')
plt.xlabel('特征值')
plt.ylabel('频率')

# BERT特征分布
plt.subplot(133)
plt.hist(bert_features.flatten(), bins=50)
plt.title('BERT特征分布')
plt.xlabel('特征值')
plt.ylabel('频率')

plt.tight_layout()
plt.show()

## 3. 用户特征提取

提取用户相关的特征，包括：
1. 基础统计特征：粉丝数、关注数、发文数等
2. 账号特征：账号年龄、是否认证等
3. 历史行为特征：发文频率、互动率等

In [ ]:
# 3.1 提取用户特征
def extract_user_features(user_info):
    if not isinstance(user_info, dict):
        return {
            'followers_count': 0,
            'friends_count': 0,
            'statuses_count': 0,
            'verified': 0,
            'account_age_days': 0
        }
    
    # 计算账号年龄（天数）
    created_at = pd.to_datetime(user_info.get('created_at', pd.Timestamp.now()))
    account_age = (pd.Timestamp.now() - created_at).days
    
    return {
        'followers_count': user_info.get('followers_count', 0),
        'friends_count': user_info.get('friends_count', 0),
        'statuses_count': user_info.get('statuses_count', 0),
        'verified': int(user_info.get('verified', False)),
        'account_age_days': account_age
    }

# 提取用户特征
print("提取用户特征...")
user_features = ced_df['user'].apply(extract_user_features)
user_features_df = pd.DataFrame(user_features.tolist())

print("\n用户特征统计：")
print(user_features_df.describe())

# 可视化用户特征分布
plt.figure(figsize=(15, 10))

# 粉丝数分布
plt.subplot(221)
plt.hist(np.log1p(user_features_df['followers_count']), bins=50)
plt.title('粉丝数分布(log)')
plt.xlabel('log(粉丝数 + 1)')
plt.ylabel('频率')

# 关注数分布
plt.subplot(222)
plt.hist(np.log1p(user_features_df['friends_count']), bins=50)
plt.title('关注数分布(log)')
plt.xlabel('log(关注数 + 1)')
plt.ylabel('频率')

# 发文数分布
plt.subplot(223)
plt.hist(np.log1p(user_features_df['statuses_count']), bins=50)
plt.title('发文数分布(log)')
plt.xlabel('log(发文数 + 1)')
plt.ylabel('频率')

# 账号年龄分布
plt.subplot(224)
plt.hist(user_features_df['account_age_days'], bins=50)
plt.title('账号年龄分布（天）')
plt.xlabel('账号年龄')
plt.ylabel('频率')

plt.tight_layout()
plt.show()

# 计算用户特征之间的相关性
correlation = user_features_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0)
plt.title('用户特征相关性矩阵')
plt.show()

## 4. 传播特征提取

分析微博的传播特征，包括：
1. 基础传播指标：转发数、评论数、点赞数
2. 传播速度特征：转发时间间隔分布
3. 传播深度特征：转发链条长度
4. 传播广度特征：不同用户群的参与度

In [ ]:
# 4.1 提取传播特征
def extract_propagation_features(repost_info):
    if not isinstance(repost_info, list):
        return {
            'repost_count': 0,
            'avg_interval': 0,
            'max_depth': 0,
            'unique_users': 0,
            'propagation_breadth': 0
        }
    
    # 转发数量
    repost_count = len(repost_info)
    
    # 转发时间间隔
    if repost_count > 1:
        times = sorted([pd.to_datetime(r['created_at']) for r in repost_info])
        intervals = [(t2 - t1).total_seconds() for t1, t2 in zip(times[:-1], times[1:])]
        avg_interval = np.mean(intervals)
    else:
        avg_interval = 0
    
    # 转发深度
    depths = [len(r.get('repost_path', '').split('/')) for r in repost_info]
    max_depth = max(depths) if depths else 0
    
    # 参与用户数
    unique_users = len(set(r.get('user_id') for r in repost_info))
    
    # 传播广度（每层的平均转发数）
    depth_counts = pd.Series(depths).value_counts()
    propagation_breadth = depth_counts.mean() if not depth_counts.empty else 0
    
    return {
        'repost_count': repost_count,
        'avg_interval': avg_interval,
        'max_depth': max_depth,
        'unique_users': unique_users,
        'propagation_breadth': propagation_breadth
    }

# 提取传播特征
print("提取传播特征...")
propagation_features = ced_df['reposts'].apply(extract_propagation_features)
propagation_features_df = pd.DataFrame(propagation_features.tolist())

print("\n传播特征统计：")
print(propagation_features_df.describe())

# 可视化传播特征
plt.figure(figsize=(15, 10))

# 转发数分布
plt.subplot(221)
plt.hist(np.log1p(propagation_features_df['repost_count']), bins=50)
plt.title('转发数分布(log)')
plt.xlabel('log(转发数 + 1)')
plt.ylabel('频率')

# 平均时间间隔分布
plt.subplot(222)
plt.hist(np.log1p(propagation_features_df['avg_interval']), bins=50)
plt.title('平均转发间隔分布(log)')
plt.xlabel('log(平均间隔(秒) + 1)')
plt.ylabel('频率')

# 最大深度分布
plt.subplot(223)
plt.hist(propagation_features_df['max_depth'], bins=50)
plt.title('最大转发深度分布')
plt.xlabel('最大深度')
plt.ylabel('频率')

# 传播广度分布
plt.subplot(224)
plt.hist(propagation_features_df['propagation_breadth'], bins=50)
plt.title('传播广度分布')
plt.xlabel('平均每层转发数')
plt.ylabel('频率')

plt.tight_layout()
plt.show()

# 分析特征与谣言标签的关系
ced_df['is_rumor'] = ced_df['label'].fillna(0)
propagation_features_df['is_rumor'] = ced_df['is_rumor']

# 计算相关性
correlation_with_label = propagation_features_df.corr()['is_rumor'].sort_values(ascending=False)
print("\n传播特征与谣言标签的相关性：")
print(correlation_with_label)

## 5. 特征整合与保存

将所有提取的特征整合到一起：
1. 合并文本、用户和传播特征
2. 特征标准化
3. 保存特征到文件
4. 特征重要性分析

In [ ]:
# 5.1 特征整合
from sklearn.preprocessing import StandardScaler

# 合并所有特征
def combine_features(text_features, user_features, propagation_features):
    # 标准化用户特征
    user_scaler = StandardScaler()
    user_features_scaled = user_scaler.fit_transform(user_features)
    
    # 标准化传播特征
    prop_scaler = StandardScaler()
    propagation_features_scaled = prop_scaler.fit_transform(propagation_features)
    
    # 合并特征
    combined_features = np.hstack([
        text_features,  # 文本特征（TF-IDF或词向量）
        user_features_scaled,  # 用户特征
        propagation_features_scaled  # 传播特征
    ])
    
    return combined_features

# 准备要合并的特征
# 这里使用TF-IDF作为文本特征的示例
print("合并特征...")
combined_features = combine_features(
    tfidf_features.toarray(),
    user_features_df.drop('account_age_days', axis=1),  # 移除高度相关的特征
    propagation_features_df.drop('is_rumor', axis=1)
)

print(f"最终特征维度: {combined_features.shape}")

# 保存特征
output_dir = Path('../results')
output_dir.mkdir(exist_ok=True)

np.save(output_dir / 'combined_features.npy', combined_features)
np.save(output_dir / 'labels.npy', ced_df['is_rumor'].values)

print("\n特征已保存到results目录")

# 特征重要性分析（使用随机森林）
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# 训练随机森林模型
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(combined_features, ced_df['is_rumor'])

# 获取特征重要性
feature_names = (
    list(tfidf.get_feature_names_out()) +
    list(user_features_df.drop('account_age_days', axis=1).columns) +
    list(propagation_features_df.drop('is_rumor', axis=1).columns)
)

feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': rf.feature_importances_
})

# 显示最重要的20个特征
print("\n最重要的20个特征：")
print(feature_importance.sort_values('importance', ascending=False).head(20))

# 可视化特征重要性
plt.figure(figsize=(12, 6))
plt.bar(range(20), feature_importance.sort_values('importance', ascending=False)['importance'][:20])
plt.xticks(range(20), feature_importance.sort_values('importance', ascending=False)['feature'][:20], rotation=45, ha='right')
plt.title('Top 20 最重要特征')
plt.tight_layout()
plt.show()